<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/equitycashflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import QuantLib as ql
import unittest
import math

# Helper utilities (assuming flat_rate_py, flat_vol_py are defined as in previous examples)
def flat_rate_py(value_or_date, day_counter_or_value=None, day_counter_if_date=None):
    if isinstance(value_or_date, ql.Date):
        eval_date = value_or_date
        rate_val = day_counter_or_value
        dc = day_counter_if_date
    else: # value, day_counter
        eval_date = ql.Settings.instance().evaluationDate
        rate_val = value_or_date
        dc = day_counter_or_value

    if isinstance(rate_val, ql.Quote):
        quote_handle = ql.QuoteHandle(rate_val)
    elif isinstance(rate_val, float):
        quote_handle = ql.QuoteHandle(ql.SimpleQuote(rate_val))
    else:
        quote_handle = rate_val # Assuming it's already a Handle<Quote>
    return ql.FlatForward(eval_date, quote_handle, dc)

def flat_vol_py(value_or_date, day_counter_or_value=None, day_counter_if_date=None):
    if isinstance(value_or_date, ql.Date):
        eval_date = value_or_date
        vol_val = day_counter_or_value
        dc = day_counter_if_date
    else:
        eval_date = ql.Settings.instance().evaluationDate
        vol_val = value_or_date
        dc = day_counter_or_value

    if isinstance(vol_val, ql.Quote):
        vol_quote_handle = ql.QuoteHandle(vol_val)
    elif isinstance(vol_val, float):
        vol_quote_handle = ql.QuoteHandle(ql.SimpleQuote(vol_val))
    else:
        vol_quote_handle = vol_val # Assuming it's already a Handle<Quote>
    return ql.BlackConstantVol(eval_date, ql.NullCalendar(), vol_quote_handle, dc)


class CommonVarsEquityCF: # Renamed to avoid conflicts
    def __init__(self):
        self.calendar = ql.TARGET()
        self.day_count = ql.Actual365Fixed()
        self.notional = 1.0e7

        self.today = self.calendar.adjust(ql.Date(27, ql.January, 2023))
        # Settings.instance().evaluationDate() is set in TestCase.setUp

        # Handles first, then link
        self.local_ccy_interest_handle = ql.RelinkableYieldTermStructureHandle()
        self.dividend_handle = ql.RelinkableYieldTermStructureHandle()
        self.quanto_ccy_interest_handle = ql.RelinkableYieldTermStructureHandle()
        self.equity_vol_handle = ql.RelinkableBlackVolTermStructureHandle()
        self.fx_vol_handle = ql.RelinkableBlackVolTermStructureHandle()
        self.spot_handle = ql.RelinkableQuoteHandle()
        self.correlation_handle = ql.RelinkableQuoteHandle()

        self.equity_index = ql.EquityIndex("eqIndex", self.calendar, ql.EURCurrency(),
                                           self.local_ccy_interest_handle,
                                           self.dividend_handle, self.spot_handle)
        self.equity_index.addFixing(ql.Date(5, ql.January, 2023), 9010.0)
        self.equity_index.addFixing(self.today, 8690.0) # Fixing for today might be spot or from history

        # Link handles after index creation (if index needs them at construction)
        # The order here is important if index constructor tries to use them.
        # Better to link before passing to index constructor if they're not optional.
        # For this test, linking after seems fine if index uses them lazily.
        self.local_ccy_interest_handle.linkTo(flat_rate_py(0.0375, self.day_count))
        self.dividend_handle.linkTo(flat_rate_py(0.005, self.day_count))
        self.quanto_ccy_interest_handle.linkTo(flat_rate_py(0.001, self.day_count))
        self.equity_vol_handle.linkTo(flat_vol_py(0.4, self.day_count))
        self.fx_vol_handle.linkTo(flat_vol_py(0.2, self.day_count))
        self.spot_handle.linkTo(ql.SimpleQuote(8700.0))
        self.correlation_handle.linkTo(ql.SimpleQuote(0.4))

    def create_equity_quanto_cash_flow(self, index, start_date=None, end_date=None, use_quanto_pricer=True):
        # Overload based on provided args
        if start_date is None and end_date is None: # Default dates
            start_date = ql.Date(5, ql.January, 2023)
            end_date = ql.Date(5, ql.April, 2023)

        # paymentDate defaults to endDate if not specified for EquityCashFlow
        cf = ql.EquityCashFlow(self.notional, index, start_date, end_date, end_date)
        if use_quanto_pricer:
            pricer = ql.EquityQuantoCashFlowPricer(
                self.quanto_ccy_interest_handle, self.equity_vol_handle,
                self.fx_vol_handle, self.correlation_handle
            )
            cf.setPricer(pricer)
        return cf

    def bump_market_data(self):
        self.local_ccy_interest_handle.linkTo(flat_rate_py(0.04, self.day_count))
        self.dividend_handle.linkTo(flat_rate_py(0.01, self.day_count))
        self.quanto_ccy_interest_handle.linkTo(flat_rate_py(0.03, self.day_count))
        self.equity_vol_handle.linkTo(flat_vol_py(0.45, self.day_count))
        self.fx_vol_handle.linkTo(flat_vol_py(0.25, self.day_count))
        self.spot_handle.linkTo(ql.SimpleQuote(8710.0)) # New SimpleQuote, then link

class EquityCashFlowTests(unittest.TestCase):
    def setUp(self):
        self.saved_eval_date = ql.Settings.instance().evaluationDate
        self.common_vars = CommonVarsEquityCF()
        ql.Settings.instance().evaluationDate = self.common_vars.today

    def tearDown(self):
        ql.Settings.instance().evaluationDate = self.saved_eval_date

    def test_simple_equity_cash_flow(self):
        print("Testing simple equity cash flow...")
        tolerance = 1.0e-6
        vars = self.common_vars

        # Create CF without quanto pricer
        cf = vars.create_equity_quanto_cash_flow(vars.equity_index, use_quanto_pricer=False)

        index_start = vars.equity_index.fixing(cf.baseDate())
        index_end = vars.equity_index.fixing(cf.fixingDate())

        expected_amount = (index_end / index_start - 1.0) * vars.notional
        actual_amount = cf.amount()

        self.assertAlmostEqual(actual_amount, expected_amount, delta=tolerance,
                               msg=f"Simple equity cash flow amount mismatch: "
                                   f"Actual: {actual_amount}, Expected: {expected_amount}")

    def _check_quanto_correction(self, include_dividend, bump_data=False):
        tolerance = 1.0e-6
        vars = self.common_vars

        if include_dividend:
            equity_index_test = vars.equity_index
        else:
            # Clone index without dividend yield (link an empty handle or a zero rate curve)
            # ql.EquityIndex.clone(forecastCurve, dividendCurve, spot)
            # If dividendCurve is empty handle, it implies zero dividends.
            empty_div_handle = ql.YieldTermStructureHandle() # Empty handle
            equity_index_test = vars.equity_index.clone(
                vars.local_ccy_interest_handle, # forecast curve for equity index
                empty_div_handle,               # dividend curve
                vars.spot_handle                # spot quote handle
            )
            # Need to ensure fixings are copied or re-added if clone doesn't preserve them
            equity_index_test.addFixing(ql.Date(5, ql.January, 2023), 9010.0)
            equity_index_test.addFixing(vars.today, 8690.0)


        cf = vars.create_equity_quanto_cash_flow(equity_index_test, use_quanto_pricer=True)

        if bump_data:
            vars.bump_market_data()

        # Ensure market data is up-to-date for formula calculation after potential bump
        # The handles within vars are updated, so these reflect current market
        strike_val = equity_index_test.fixing(cf.fixingDate()) # Fixing at fixingDate is S_T effectively
        index_start_val = equity_index_test.fixing(cf.baseDate()) # S_0

        # Get time from reference date of the interest rate curve
        time_to_fixing = vars.local_ccy_interest_handle.timeFromReference(cf.fixingDate())

        rf = vars.local_ccy_interest_handle.zeroRate(time_to_fixing, ql.Continuous).rate()
        q_div = 0.0
        if include_dividend and not vars.dividend_handle.empty():
             q_div = vars.dividend_handle.zeroRate(time_to_fixing, ql.Continuous).rate()

        eq_vol = vars.equity_vol_handle.blackVol(cf.fixingDate(), strike_val) # Vol at fixing date and strike
        fx_vol = vars.fx_vol_handle.blackVol(cf.fixingDate(), 1.0) # FX vol (strike usually 1 for FX)
        rho = vars.correlation_handle.value()
        spot_val = vars.spot_handle.value() # Current spot of the index

        # The quanto forward formula: S_t * exp((r_local - q_equity - rho * sigma_equity * sigma_fx) * T)
        # Here, cf.fixingDate() is S_T, which is what we are trying to match.
        # The test seems to imply that the EquityCashFlow.amount() for a QUANTO flow gives
        # (QuantoFwd / S_initial - 1) * Notional.
        # Where QuantoFwd is the forward price of the equity index in the *foreign* (quanto) currency terms,
        # adjusted for correlation.
        # The formula given in C++ is: spot * exp((rf - q - rho * eqVol * fxVol) * time)
        # This is the standard quanto forward formula where 'spot' is the *current* spot S_t (at eval date).
        # The EquityCashFlow's amount is (S_T / S_0 - 1) * Notional, where S_T needs the quanto adjustment.
        # S_T (quanto adjusted) = S_0_fixing * exp((r_local - q_equity - rho * sigma_equity * sigma_fx) * T_period + (r_local - q_equity) * T_drift_only_part_if_any)
        # This seems like the test is comparing the CF amount using the pricer against a manually calculated
        # quanto-adjusted forward.

        # Let's use spot_val as S(eval_date) to calculate the expected quanto forward at fixingDate.
        # The time 'T' in the formula is from eval_date to fixing_date.
        # The EquityCashFlow is based on fixings S(baseDate) and S(fixingDate).
        # If fixingDate is in the future, S(fixingDate) is the forward.
        # If fixingDate is in the past or today, S(fixingDate) is a known fixing.

        # The expected amount in the C++ test:
        # quantoForward = spot * std::exp((rf - q - rho * eqVol * fxVol) * time);
        # expectedAmount = (quantoForward / indexStart - 1.0) * vars.notional;
        # Here 'spot' is vars.spotHandle->value(), which is S(eval_date).
        # 'indexStart' is equityIndex->fixing(cf->baseDate()).
        # 'time' is localCcyInterestHandle->timeFromReference(cf->fixingDate()), this is time from eval_date to fixing_date.

        # This implies the expected amount is trying to replicate what the pricer does if
        # the fixing at fixingDate were a forward calculated today.
        # However, the pricer for EquityCashFlow typically uses the actual fixing S(fixingDate)
        # and then discounts its payoff S(fixingDate)/S(baseDate) - 1, with quanto adjustment applied to the discount factor.

        # Let's re-evaluate the C++ test logic for "expectedAmount".
        # The EquityQuantoCashFlowPricer should value: DiscountFactor_Quanto * (S_T/S_0 - 1) * Notional
        # where S_T and S_0 are actual fixings. The quanto adjustment affects the discount factor.
        # D_quanto(T) = D_local(T) * exp(-rho * sigma_eq * sigma_fx * T)
        # The C++ test seems to be calculating an *expected future spot* using the quanto drift,
        # and then forms a payoff based on that future spot. This is unusual for a cashflow pricer.
        # A standard EquityCashFlow.amount() returns the undiscounted payoff (S_T/S_0 - 1) * N.
        # If a pricer is set, pricer.swapletPrice() would give the PV.
        # The C++ test calls cf->amount(). This usually returns the nominal undiscounted amount.
        # It's possible the EquityQuantoCashFlowPricer overrides `amount()` to return a forward-like value
        # if the fixingDate is in the future.

        # Let's follow the C++ formula for `expectedAmount` strictly for now.
        # `time` is from eval_date (vars.today) to cf.fixingDate()
        time_from_eval_to_fixing = vars.local_ccy_interest_handle.timeFromReference(cf.fixingDate())

        # If fixing date is today or in the past, quantoForward should just be the fixing
        if cf.fixingDate() <= vars.today:
            quanto_forward_expected = equity_index_test.fixing(cf.fixingDate())
        else:
            quanto_forward_expected = spot_val * math.exp(
                (rf - q_div - rho * eq_vol * fx_vol) * time_from_eval_to_fixing
            )

        expected_amount = (quanto_forward_expected / index_start_val - 1.0) * vars.notional
        actual_amount = cf.amount() # This is the core of the test.

        self.assertAlmostEqual(actual_amount, expected_amount, delta=tolerance * abs(vars.notional) if vars.notional != 0 else tolerance,
                               msg=f"Quanto correction mismatch (includeDividend={include_dividend}, bumpData={bump_data}):\n"
                                   f"Actual: {actual_amount}, Expected: {expected_amount}\n"
                                   f"Index Start: {index_start_val}, QuantoFwd: {quanto_forward_expected}, Spot: {spot_val}\n"
                                   f"rf={rf}, q={q_div}, eqVol={eq_vol}, fxVol={fx_vol}, rho={rho}, T={time_from_eval_to_fixing}")


    def test_quanto_correction(self):
        print("Testing quanto correction...")
        self._check_quanto_correction(True, False)  # With dividend, no bump
        self._check_quanto_correction(False, False) # No dividend, no bump
        self._check_quanto_correction(False, True)  # No dividend, with bump (observability)

    def test_error_when_base_date_after_fixing_date(self):
        print("Testing error when base date after fixing date...")
        vars = self.common_vars
        end_date = ql.Date(5, ql.January, 2023)
        start_date = ql.Date(5, ql.April, 2023) # Start after end

        # Error should be raised at construction or when amount() is called if validation is there.
        # QuantLib often raises at construction for invalid date orderings.
        with self.assertRaisesRegex(RuntimeError, "fixing date.*before base date"):
            cf = vars.create_equity_quanto_cash_flow(vars.equity_index, start_date, end_date)
            # The error might also occur at cf.amount() if construction is lenient.
            # cf.amount() # If needed to trigger error.

    def _check_error_on_empty_handle(self, handle_attr_name, error_message_substr):
        vars = self.common_vars
        cf = vars.create_equity_quanto_cash_flow(vars.equity_index, use_quanto_pricer=True)

        original_link = getattr(vars, handle_attr_name).linkedTo() # Get current linked object
        empty_ts_handle = type(getattr(vars, handle_attr_name))() # Create empty handle of same type

        try:
            getattr(vars, handle_attr_name).linkTo(empty_ts_handle.linkedTo()) # Link to empty target
            with self.assertRaisesRegex(RuntimeError, error_message_substr):
                cf.amount() # Error should occur here
        finally:
            getattr(vars, handle_attr_name).linkTo(original_link) # Restore

    def test_error_when_quanto_curve_handle_is_empty(self):
        print("Testing error when quanto currency curve handle is empty...")
        self._check_error_on_empty_handle("quanto_ccy_interest_handle", "Quanto currency term structure handle cannot be empty")

    def test_error_when_equity_vol_handle_is_empty(self):
        print("Testing error when equity vol handle is empty...")
        self._check_error_on_empty_handle("equity_vol_handle", "Equity volatility term structure handle cannot be empty")

    def test_error_when_fx_vol_handle_is_empty(self):
        print("Testing error when FX vol handle is empty...")
        self._check_error_on_empty_handle("fx_vol_handle", "FX volatility term structure handle cannot be empty")

    def test_error_when_correlation_handle_is_empty(self):
        print("Testing error when correlation handle is empty...")
        vars = self.common_vars
        cf = vars.create_equity_quanto_cash_flow(vars.equity_index, use_quanto_pricer=True)

        original_link = vars.correlation_handle.linkedTo()
        # For QuoteHandle, linking to an empty SimpleQuote or just an empty handle
        empty_quote = ql.SimpleQuote() # An empty quote has value Null<Real>
        # vars.correlation_handle.linkTo(empty_quote) # This sets value to Null
        # C++ test links to ext::shared_ptr<Quote>() which is a null pointer.
        # Python equivalent for RelinkableQuoteHandle: link to a new SimpleQuote that has no value,
        # or more directly, ensure the handle is considered "empty" by the pricer.
        # A SimpleQuote() is not "empty" in the handle sense.
        # Let's try linking to a totally new, unlinked handle's target if that's how QL checks.
        # Or, the check might be on `correlationHandle->value()` failing.
        # If we want to test if the handle itself is "empty" in a null pointer sense, that's harder.
        # Let's try unlinking:
        vars.correlation_handle.linkTo(ql.Quote()) # Link to a default-constructed (empty) Quote
                                                     # This is the closest to a null shared_ptr<Quote>

        try:
            with self.assertRaisesRegex(RuntimeError, "Correlation handle cannot be empty"):
                cf.amount()
        finally:
            vars.correlation_handle.linkTo(original_link)


    def test_error_when_inconsistent_market_data_reference_date(self):
        print("Testing error when market data reference dates are inconsistent...")
        vars = self.common_vars
        cf = vars.create_equity_quanto_cash_flow(vars.equity_index, use_quanto_pricer=True)

        original_link = vars.quanto_ccy_interest_handle.linkedTo()
        # Create a curve with a different reference date
        different_ref_date = vars.today - ql.Period(1, ql.Days)
        vars.quanto_ccy_interest_handle.linkTo(
            flat_rate_py(different_ref_date, 0.02, vars.day_count)
        )
        try:
            with self.assertRaisesRegex(RuntimeError, "need to have the same reference date"):
                cf.amount()
        finally:
            vars.quanto_ccy_interest_handle.linkTo(original_link)


if __name__ == '__main__':
    print("Testing QuantLib " + ql.__version__)
    unittest.main(argv=['first-arg-is-ignored'], exit=False)